In [33]:
import yfinance as yf
import numpy as np
from newsapi import NewsApiClient
from datetime import datetime, timedelta

In [ ]:
# -------- CONFIG --------
SYMBOL = "GOLDBEES.NS"   # change to SILVERBEES.NS / URA
NEWS_API_KEY = "your_news_api_key_here"

In [35]:
# -------- MARKET DATA --------
def fetch_data(symbol):
    df = yf.download(symbol, period="6mo", interval="1d", progress=False)
    df.dropna(inplace=True)
    return df

In [36]:
# -------- AUTO SUPPORT (20-day swing low avg) --------
def auto_support(df, window=20):
    recent = df.tail(window)
    return recent["Low"].mean()

In [37]:
# -------- NEWS CHECK --------
def check_bad_news(symbol):
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)

    keyword_map = {
        "GOLDBEES.NS": "gold commodities",
        "SILVERBEES.NS": "silver commodities",
        "URA": "uranium nuclear energy"
    }

    q = keyword_map.get(symbol, "commodities")

    from_date = (datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")

    articles = newsapi.get_everything(
        q=q,
        sources="reuters,bloomberg,wsj",
        from_param=from_date,
        language="en",
        sort_by="relevancy",
        page_size=10,
    )

    long_term_words = [
        "ban", "shutdown", "collapse", "structural",
        "regulation", "permanent", "demand destruction"
    ]

    for a in articles["articles"]:
        text = (a["title"] + str(a["description"])).lower()
        if any(w in text for w in long_term_words):
            return True, a["title"]

    return False, None


In [38]:
# -------- DECISION ENGINE --------
def decide(df, support, bad_news):
    today = df.iloc[-1]
    yesterday = df.iloc[-2]

    # Extract scalar values using .item() for Series
    close_today = today["Close"].item() if hasattr(today["Close"], 'item') else today["Close"]
    close_yesterday = yesterday["Close"].item() if hasattr(yesterday["Close"], 'item') else yesterday["Close"]
    volume_today = today["Volume"].item() if hasattr(today["Volume"], 'item') else today["Volume"]
    
    change_pct = ((close_today - close_yesterday) / close_yesterday) * 100

    avg_volume = df["Volume"].tail(20).mean()
    high_volume_result = volume_today > 1.5 * avg_volume
    high_volume = high_volume_result.item() if hasattr(high_volume_result, 'item') else high_volume_result
    
    support_holds_result = close_today >= support
    support_holds = support_holds_result.item() if hasattr(support_holds_result, 'item') else support_holds_result

    if bad_news[0]:
        return f"AVOID ❌ | {bad_news[1]}"

    if not support_holds and high_volume:
        return "WAIT ⏸️ | Support broken with volume"

    if change_pct <= -3 and support_holds:
        return "BUY ✅ | Strong dip at support"

    if -3 < change_pct < -1 and support_holds:
        return "BUY SMALL ⚠️ | Mild dip"

    return "WAIT ⏸️ | No edge today"

In [40]:
# -------- RUN FOR ALL SYMBOLS --------
import pandas as pd


STOCKS = [
 "SUNPHARMA.NS","DRREDDY.NS","CIPLA.NS","DIVISLAB.NS","AUROPHARMA.NS",
 "HAL.NS","BEL.NS","BDL.NS","MAZDOCK.NS",
 "DIXON.NS","KAYNES.NS","AMBER.NS","PGEL.NS",
 "NTPC.NS","POWERGRID.NS","TATAPOWER.NS",
 "TCS.NS","INFY.NS",
 "HDFCBANK.NS","ICICIBANK.NS","GOLDBEES.NS","SILVERBEES.NS","URA"
]


results = []

for symbol in STOCKS:
    
    df = fetch_data(symbol)
    support = auto_support(df)
    bad_news = check_bad_news(symbol)
    decision = decide(df, support, bad_news)
    
    # Extract scalar values for clean output
    last_close = df.iloc[-1]['Close'].item() if hasattr(df.iloc[-1]['Close'], 'item') else df.iloc[-1]['Close']
    support_val = support.item() if hasattr(support, 'item') else support
    
    results.append({
        'ETF': symbol,
        'Last Close': round(last_close, 2),
        'Auto Support': round(support_val, 2),
        'Decision': decision
    })

# Create and display table
results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("METALS ETF ANALYSIS RESULTS")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)


METALS ETF ANALYSIS RESULTS
          ETF  Last Close  Auto Support                             Decision
 SUNPHARMA.NS     1610.60       1678.64 WAIT ⏸️ | Support broken with volume
   DRREDDY.NS     1222.50       1206.97              BUY SMALL ⚠️ | Mild dip
     CIPLA.NS     1328.40       1418.23              WAIT ⏸️ | No edge today
  DIVISLAB.NS     6188.00       6260.70              WAIT ⏸️ | No edge today
AUROPHARMA.NS     1139.90       1164.51              WAIT ⏸️ | No edge today
       HAL.NS     4624.00       4379.71              WAIT ⏸️ | No edge today
       BEL.NS      453.00        407.54              WAIT ⏸️ | No edge today
       BDL.NS     1570.00       1476.96              WAIT ⏸️ | No edge today
   MAZDOCK.NS     2505.60       2416.24              WAIT ⏸️ | No edge today
     DIXON.NS    10279.00      11104.35              WAIT ⏸️ | No edge today
    KAYNES.NS     3490.60       3623.90              WAIT ⏸️ | No edge today
     AMBER.NS     5614.00       6097.25        